# PINN Implementation of Ashourvan & Diamond Paper

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import grad

import mlflow
import mlflow.pytorch

from tqdm.notebook import tqdm

import imageio.v2 as imageio
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import os

## Define Parameters

In [2]:
l = 1.0     # not provided by paper, need to check relation with Λ 
α = 6.0
D_c = 0.78
C_χ = 0.95
a_u = 1.0
μ_c = 0.78
β = 0.1
Λ = 4000.0
ϵ_c = 6.25

In [3]:
g_i = 5.1
ϵ_i = 0.002

In [4]:
physical_params = {
    'l': l,
    'α': α,
    'D_c': D_c,
    'C_χ': C_χ,
    'a_u': a_u,
    'μ_c': μ_c,
    'β': β,
    'Λ': Λ,
    'ϵ_c': ϵ_c,
    'g_i': g_i,
    'ϵ_i': ϵ_i
}

## Define PDEs

### Define repeated calculations

In [5]:
def compute_w(C_χ, l, ϵ, α, a_u, u):
    return C_χ * l**2 * ϵ / torch.sqrt(α**2 + a_u * u**2)

### Dynamic equation for mean density


In [6]:
def pde_mean_density(x, n_t, n_x, n_xx, ϵ, l, α, D_c):    
    intermediate = l**2 * ϵ * n_x / α
    intermediate_x = grad(intermediate, x, grad_outputs=torch.ones_like(intermediate), retain_graph=True, create_graph=True)[0]
    
    return (n_t - intermediate_x - D_c * n_xx)**2

### Dynamic equation for mean vorticity

In [7]:
def pde_mean_vorticity(x, ϵ, n_x, u_t, u_xx, l, α, μ_c, w):    
    intermediate = (l**2 * ϵ / α - w) * n_x
    intermediate_x = grad(intermediate, x, grad_outputs=torch.ones_like(intermediate), retain_graph=True, create_graph=True)[0]
    
    return (u_t - intermediate_x - w * u_xx - μ_c * u_xx)**2

### Dynamic equation for turbulent potential entrosphy

In [8]:
def pde_tpe(x, ϵ, n_x, u_x, ϵ_t, ϵ_x, l, β, Λ, ϵ_c, w):
    intermediate = l**2 * torch.sqrt(ϵ) * ϵ_x
    intermediate_x = grad(intermediate, x, grad_outputs=torch.ones_like(intermediate), retain_graph=True, create_graph=True)[0]
    
    return (ϵ_t - β * intermediate_x - Λ * (w * (n_x - u_x)**2 - ϵ**(3/2) / ϵ_c**0.5 + ϵ))**2

## Define Initial Conditions

In [9]:
def n_initial_cond(t, x):
    return -g_i * x

def u_initial_cond(t, x):
    return torch.zeros(x.shape, device=x.device.type)

def ϵ_initial_cond(t, x):
    return torch.full(x.shape, ϵ_i, device=x.device.type)

## Define Boundary Conditions

In [10]:
def n_boundary_cond(t, x):
    out = torch.full(x.shape, -g_i, device=x.device.type)
    out = out * x
    
    return out

def u_boundary_cond(t, x):
    return torch.zeros(x.shape, device=x.device.type)


def ϵ_x_boundary_cond(t, x):
    return torch.zeros(x.shape, device=x.device.type)

## PINN Implementation

In [11]:
class CustomOutputLayer(nn.Module):
    def __init__(self, pars):
        super(CustomOutputLayer, self).__init__()
        self.n = nn.Sequential(nn.Linear(pars['width'], 1))
        self.u = nn.Sequential(nn.Linear(pars['width'], 1))
        self.ϵ = nn.Sequential(nn.Linear(pars['width'], 1))
        
    def forward(self, x):
        return torch.hstack((self.n(x), self.u(x), torch.abs(self.ϵ(x) + 1e-2)))

In [12]:
class PINN(nn.Module):
    def __init__(self, pars: dict):
        super().__init__()
        self.pars = pars
        
        self.modules = [nn.BatchNorm1d(2), nn.Linear(2, self.pars['width'])] # nn.LayerNorm(2)
        for i in range(self.pars['layers'] - 1):
            # self.modules.append(nn.LayerNorm(self.pars['width']))
            self.modules.append(nn.GELU())
            self.modules.append(nn.Linear(self.pars['width'], self.pars['width']))
        
        # self.modules.append(nn.LayerNorm(self.pars['width']))
        self.modules.append(CustomOutputLayer(pars))
        
        self.model = nn.Sequential(*self.modules)
        self.model.to(self.pars['device'])
        
        self.optimizer = torch.optim.Adam(params=self.model.parameters(), lr=self.pars['lr'])
        self.num_params = sum([len(params) for params in [p for p in self.model.parameters()]])
        
        self.epoch = 0
        
        t = np.linspace(self.pars['t_min'], self.pars['t_max'], 100)
        x = np.linspace(self.pars['x_min'], self.pars['x_max'], 100)
        
        self.stage_interval = self.pars['epochs']//self.pars['num_stages']

        self.eval_t, self.eval_x = np.meshgrid(t, x)
        self.eval_t = torch.Tensor(self.eval_t).reshape(-1, 1).to(self.pars['device'])
        self.eval_x = torch.Tensor(self.eval_x).reshape(-1, 1).to(self.pars['device'])
        
        self.eval_t.requires_grad_()
        self.eval_x.requires_grad_()
        
        eval_t_interior, eval_x_interior = np.meshgrid(t[1:-1], x[1:-1])
        eval_t_interior = torch.Tensor(eval_t_interior).reshape(-1, 1).to(self.pars['device'])
        eval_x_interior = torch.Tensor(eval_x_interior).reshape(-1, 1).to(self.pars['device'])
        
        self.eval_X_interior = torch.hstack((eval_t_interior, eval_x_interior))
        self.eval_X_interior.requires_grad_()
        
        eval_t_initial = torch.zeros_like(self.eval_x)
        self.eval_X_initial = torch.hstack((eval_t_initial, self.eval_x))
        self.eval_X_initial.requires_grad_()
        
        eval_t_boundary = torch.Tensor(np.vstack([t, t])).reshape(-1, 1).to(self.pars['device'])
        eval_x_boundary = torch.Tensor(np.vstack([x[0] * np.ones_like(t), x[-1] * np.ones_like(t)])).reshape(-1, 1).to(self.pars['device'])
        
        self.eval_X_boundary = torch.hstack((eval_t_boundary, eval_x_boundary))
        self.eval_X_boundary.requires_grad_()
        
        self.plot_t = self.eval_t.view(100, 100).detach().cpu().numpy()
        self.plot_x = self.eval_x.view(100, 100).detach().cpu().numpy()
        
        self.plot_files = {}
        
        if 'plot_vars_list' in self.pars:
            for varname in self.pars['plot_vars_list']:
                self.plot_files[varname] = []
        
    def __call__(self, X):
        return self.model(X)
        
    def sample_interior_points(self, percent):
        t = torch.empty((self.pars['interior_batch_size'], 1), device=self.pars['device']).uniform_(self.pars['t_min'], percent * self.pars['t_max'])
        x1 = torch.empty((self.pars['interior_batch_size']//2, 1), device=self.pars['device']).uniform_(self.pars['x_min'], self.pars['x_min'] + percent * (self.pars['x_max'] - self.pars['x_min']))
        x2 = torch.empty((self.pars['interior_batch_size']//2, 1), device=self.pars['device']).uniform_(self.pars['x_min'] + (1 - percent) * (self.pars['x_max'] - self.pars['x_min']), self.pars['x_max'])
        x = torch.vstack((x1, x2))
        X_interior = torch.cat((t, x), 1)
        X_interior.requires_grad_()
        
        return X_interior
    
    def sample_initial_points(self):
        t = torch.zeros(self.pars['initial_batch_size'], 1, device=self.pars['device'])
        x = torch.empty((self.pars['initial_batch_size'], 1), device=self.pars['device']).uniform_(self.pars['x_min'], self.pars['x_max'])
        X_initial = torch.cat((t, x), 1)
        X_initial.requires_grad_()
        
        return X_initial
    
    def sample_boundary_points(self):
        options = torch.tensor([self.pars['x_min'], self.pars['x_max']], device=self.pars['device'])
        
        t = torch.empty((self.pars['boundary_batch_size'], 1), device=self.pars['device']).uniform_(self.pars['t_min'], self.pars['t_max'])
        x = options[torch.randint(0, 2, (self.pars['boundary_batch_size'], 1), device=self.pars['device'])]
        X_boundary = torch.cat((t, x), 1)
        X_boundary.requires_grad_()
        
        return X_boundary
    
    def forward(self, X_interior, X_initial, X_boundary):
        # X shape: (batch_size, 2), where 2nd dimension is [t, x]
        # Y shape: (batch_size, 3), where 2nd dimension is [n, u, ϵ]
        
        t_interior = X_interior[:, 0].reshape(-1, 1)
        x_interior = X_interior[:, 1].reshape(-1, 1)
        t_initial = X_initial[:, 0].reshape(-1, 1)
        x_initial = X_initial[:, 1].reshape(-1, 1)
        t_boundary = X_boundary[:, 0].reshape(-1, 1)
        x_boundary = X_boundary[:, 1].reshape(-1, 1)
        
        # forward pass
        Y_interior = self.model(torch.hstack((t_interior, x_interior)))
        Y_initial = self.model(torch.hstack((t_initial, x_initial)))
        Y_boundary = self.model(torch.hstack((t_boundary, x_boundary)))
        
        n_interior = Y_interior[:, 0].reshape(-1, 1)
        u_interior = Y_interior[:, 1].reshape(-1, 1)
        ϵ_interior = Y_interior[:, 2].reshape(-1, 1)
        
        n_initial = Y_initial[:, 0].reshape(-1, 1)
        u_initial = Y_initial[:, 1].reshape(-1, 1)
        ϵ_initial = Y_initial[:, 2].reshape(-1, 1)
        
        n_boundary = Y_boundary[:, 0].reshape(-1, 1)
        u_boundary = Y_boundary[:, 1].reshape(-1, 1)
        ϵ_boundary = Y_boundary[:, 2].reshape(-1, 1)
        
        n_x_interior = grad(n_interior, x_interior, grad_outputs=torch.ones_like(n_interior), retain_graph=True, create_graph=True)[0]
        n_t_interior = grad(n_interior, t_interior, grad_outputs=torch.ones_like(n_interior), retain_graph=True, create_graph=True)[0]
        
        n_xx_interior = grad(n_x_interior, x_interior, grad_outputs=torch.ones_like(n_x_interior), retain_graph=True, create_graph=True)[0]
        
        u_x_interior = grad(u_interior, x_interior, grad_outputs=torch.ones_like(u_interior), retain_graph=True, create_graph=True)[0]
        u_t_interior = grad(u_interior, t_interior, grad_outputs=torch.ones_like(u_interior), retain_graph=True, create_graph=True)[0]
        
        u_xx_interior = grad(u_x_interior, x_interior, grad_outputs=torch.ones_like(u_x_interior), retain_graph=True, create_graph=True)[0]
        
        ϵ_x_interior = grad(ϵ_interior, x_interior, grad_outputs=torch.ones_like(ϵ_interior), retain_graph=True, create_graph=True)[0]
        ϵ_t_interior = grad(ϵ_interior, t_interior, grad_outputs=torch.ones_like(ϵ_interior), retain_graph=True, create_graph=True)[0]
        
        ϵ_x_boundary = grad(ϵ_boundary, x_boundary, grad_outputs=torch.ones_like(ϵ_boundary), retain_graph=True, create_graph=True)[0]
        
        w = compute_w(C_χ, l, ϵ_interior, α, a_u, u_interior)
        
        density_loss = pde_mean_density(x_interior, n_t_interior, n_x_interior, n_xx_interior, ϵ_interior, l, α, D_c).mean()
        vorticity_loss = pde_mean_vorticity(x_interior, ϵ_interior, n_x_interior, u_t_interior, u_xx_interior, l, α, μ_c, w).mean()
        tpe_loss = pde_tpe(x_interior, ϵ_interior, n_x_interior, u_x_interior, ϵ_t_interior, ϵ_x_interior, l, β, Λ, ϵ_c, w).mean()
        interior_loss = (density_loss + vorticity_loss + tpe_loss)/3
        
        mse = nn.MSELoss()
        
        initial_n_loss = mse(n_initial_cond(t_initial, x_initial), n_initial)
        initial_u_loss = mse(u_initial_cond(t_initial, x_initial), u_initial)
        initial_ϵ_loss = mse(ϵ_initial_cond(t_initial, x_initial), ϵ_initial)
        initial_loss = (initial_n_loss + initial_u_loss + initial_ϵ_loss)/3
        
        boundary_n_loss = mse(n_boundary_cond(t_boundary, x_boundary), n_boundary)
        boundary_u_loss = mse(u_boundary_cond(t_boundary, x_boundary), u_boundary)
        boundary_ϵ_loss = mse(ϵ_x_boundary_cond(t_boundary, x_boundary), ϵ_x_boundary)
        boundary_loss = (boundary_n_loss + boundary_u_loss + boundary_ϵ_loss)/3
        total_loss = interior_loss + initial_loss + boundary_loss
        
        return total_loss, density_loss, vorticity_loss, tpe_loss, initial_n_loss, initial_u_loss, initial_ϵ_loss, boundary_n_loss, boundary_u_loss, boundary_ϵ_loss
    
    def train(self):
        cwd = os.getcwd()
        plot_dirs = os.path.join(cwd, f'plots/{self.pars["experiment_name"]}')

        if not os.path.isdir(plot_dirs):
            os.makedirs(plot_dirs)
        
        mlflow.set_experiment(self.pars['experiment_name'])
        mlflow.start_run()
        
        mlflow.log_param("physical_params", physical_params)
        mlflow.log_param("model_params", self.pars)
        
        
        stage_percent = 0
        for epoch in tqdm(range(self.pars['epochs']), position=0, leave=True, desc='Training...'): 
            self.epoch = epoch
            
            # eval
            if epoch % self.pars['eval_interval'] == 0 or epoch == self.pars['epochs'] - 1:
                
                loss, density_loss, vorticity_loss, tpe_loss, initial_n_loss, initial_u_loss, initial_ϵ_loss, boundary_n_loss, boundary_u_loss, boundary_ϵ_loss \
                    = self.forward(self.eval_X_interior, self.eval_X_initial, self.eval_X_boundary)
                
                print()
                print(f'Epoch: {self.epoch}, Loss: {loss.item():,.4e}')
                print(f"density_loss: {density_loss.item():.4e}, vorticity_loss: {vorticity_loss.item():.4e}, tpe_loss: {tpe_loss.item():.4e}")
                print(f"initial_n_loss: {initial_n_loss.item():.4e}, initial_u_loss: {initial_u_loss.item():.4e}, initial_ϵ_loss: {initial_ϵ_loss.item():.4e}")
                print(f"boundary_n_loss: {boundary_n_loss.item():.4e}, boundary_u_loss: {boundary_u_loss.item():.4e}, boundary_ϵ_loss: {boundary_ϵ_loss.item():.4e}")
                
                mlflow.log_metric("total_loss", loss.item(), step=self.epoch)
                mlflow.log_metric("density_loss", density_loss.item(), step=self.epoch)
                mlflow.log_metric("vorticity_loss", vorticity_loss.item(), step=self.epoch)
                mlflow.log_metric("tpe_loss", tpe_loss.item(), step=self.epoch)
                mlflow.log_metric("initial_n_loss", initial_n_loss.item(), step=self.epoch)
                mlflow.log_metric("initial_u_loss", initial_u_loss.item(), step=self.epoch)
                mlflow.log_metric("initial_ϵ_loss", initial_ϵ_loss.item(), step=self.epoch)
                mlflow.log_metric("boundary_n_loss", boundary_n_loss.item(), step=self.epoch)
                mlflow.log_metric("boundary_u_loss", boundary_u_loss.item(), step=self.epoch)
                mlflow.log_metric("boundary_ϵ_loss", boundary_ϵ_loss.item(), step=self.epoch)
                
                mlflow.pytorch.log_model(self.model, f"{self.pars['experiment_name']}_model_epoch_{self.epoch}")
                
                if self.pars['plot_training_outputs']:
                    self.plot_outputs()
            
            # training step
            self.optimizer.zero_grad()
            if epoch % self.stage_interval == 0:
                stage_percent += 1/self.pars['num_stages']
                print(f"Training PINN with {stage_percent*100:.2f}% of interior points")
                X_interior = self.sample_interior_points(stage_percent)
                X_initial = self.sample_initial_points()
                X_boundary = self.sample_boundary_points()
            loss, density_loss, vorticity_loss, tpe_loss, initial_n_loss, initial_u_loss, initial_ϵ_loss, boundary_n_loss, boundary_u_loss, boundary_ϵ_loss = self.forward(X_interior, X_initial, X_boundary)
            loss.backward()
            self.optimizer.step()
            
                
            if loss.isnan():
                print(f'Epoch: {self.epoch}, Loss: {loss.item():,.4e}')
                print(f"density_loss: {density_loss.item():.4e}, vorticity_loss: {vorticity_loss.item():.4e}, tpe_loss: {tpe_loss.item():.4e}")
                print(f"initial_n_loss: {initial_n_loss.item():.4e}, initial_u_loss: {initial_u_loss.item():.4e}, initial_ϵ_loss: {initial_ϵ_loss.item():.4e}")
                print(f"boundary_n_loss: {boundary_n_loss.item():.4e}, boundary_u_loss: {boundary_u_loss.item():.4e}, boundary_ϵ_loss: {boundary_ϵ_loss.item():.4e}")
                print("loss is NaN, stopping training...")
                break
            
        mlflow.pytorch.log_model(self.model, f"{self.pars['experiment_name']}_model_final")
        
        self.save_gif()
        
        mlflow.end_run()
    
    def plot_outputs(self):
        Y = self(torch.hstack((self.eval_t, self.eval_x)))

        n = Y[:, 0].view(-1, 1)
        u = Y[:, 1].view(-1, 1)
        ϵ = Y[:, 2].view(-1, 1)
        
        n_x = grad(n, self.eval_x, grad_outputs=torch.ones_like(n), retain_graph=True, create_graph=True)[0]
        n_t = grad(n, self.eval_t, grad_outputs=torch.ones_like(n), retain_graph=True)[0]
        
        n_xx = grad(n_x, self.eval_x, grad_outputs=torch.ones_like(n_x), retain_graph=True)[0]
        
        u_x = grad(u, self.eval_x, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
        u_t = grad(u, self.eval_t, grad_outputs=torch.ones_like(u), retain_graph=True)[0]
        
        u_xx = grad(u_x, self.eval_x, grad_outputs=torch.ones_like(u_x), retain_graph=True)[0]
        
        ϵ_x = grad(ϵ, self.eval_x, grad_outputs=torch.ones_like(ϵ), retain_graph=True)[0]
        ϵ_t = grad(ϵ, self.eval_t, grad_outputs=torch.ones_like(ϵ), retain_graph=True)[0]
        
        n = n.view(100, 100).detach().cpu().numpy()
        n_x = n_x.view(100, 100).detach().cpu().numpy()
        n_t = n_t.view(100, 100).detach().cpu().numpy()
        n_xx = n_xx.view(100, 100).detach().cpu().numpy()
        
        u = u.view(100, 100).detach().cpu().numpy()
        u_x = u_x.view(100, 100).detach().cpu().numpy()
        u_t = u_t.view(100, 100).detach().cpu().numpy()
        u_xx = u_xx.view(100, 100).detach().cpu().numpy()
        
        ϵ = ϵ.view(100, 100).detach().cpu().numpy()
        ϵ_x = ϵ_x.view(100, 100).detach().cpu().numpy()
        ϵ_t = ϵ_t.view(100, 100).detach().cpu().numpy()
        
        if 'n' in self.pars['plot_vars_list']:
            self.plot_var(n, 'n')
        
        if 'n_x' in self.pars['plot_vars_list']:
            self.plot_var(n_x, 'n_x')
        
        if 'n_t' in self.pars['plot_vars_list']:
            self.plot_var(n_t, 'n_t')
        
        if 'n_xx' in self.pars['plot_vars_list']:
            self.plot_var(n_xx, 'n_xx')
        
        if 'u' in self.pars['plot_vars_list']:
            self.plot_var(u, 'u')
        
        if 'u_x' in self.pars['plot_vars_list']:
            self.plot_var(u_x, 'u_x')
        
        if 'u_t' in self.pars['plot_vars_list']:
            self.plot_var(u_t, 'u_t')
        
        if 'u_xx' in self.pars['plot_vars_list']:
            self.plot_var(u_xx, 'u_xx')
        
        if 'ϵ' in self.pars['plot_vars_list']:
            self.plot_var(ϵ, 'ϵ')
        
        if 'ϵ_x' in self.pars['plot_vars_list']:
            self.plot_var(ϵ_x, 'ϵ_x')
        
        if 'ϵ_t' in self.pars['plot_vars_list']:
            self.plot_var(ϵ_t, 'ϵ_t')
        
    def plot_var(self, var, varname):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_surface(self.plot_t, self.plot_x, var, cmap='viridis')
        ax.set_xlabel('t')
        ax.set_ylabel('x')
        ax.set_zlabel(varname)
        plt.title(f'Model output {varname}, epoch {self.epoch}')
        filename = f"plots/{self.pars['experiment_name']}/plot_{varname}_epoch_{self.epoch}.png"
        self.plot_files[varname].append(filename)
        fig.savefig(filename)
        mlflow.log_artifact(filename)
        plt.close()
        
    def save_gif(self):
        for varname in self.pars['plot_vars_list']:
            gif_filename = f"plots/{self.pars['experiment_name']}/plot_{varname}.gif"
            with imageio.get_writer(gif_filename, mode='I') as writer:
                for filename in self.plot_files[varname]:
                    image = imageio.imread(filename)
                    writer.append_data(image)
                
            mlflow.log_artifact(gif_filename)
    
        

## Experiments

In [13]:
pars = {
    'experiment_name': 'simple_pinn_multistage_training_v1',
    'layers': 4,
    'width': 64,
    'lr': 1e-4,
    'epochs': 50000,
    'eval_interval': 5000,
    'interior_batch_size': 2048,
    'initial_batch_size': 2048,
    'boundary_batch_size': 2048,
    'x_min': 0.0,
    'x_max': 1.0,
    't_min': 0.0,
    't_max': 10000,
    'device': 'cuda',
    'plot_training_outputs': True,
    'plot_vars_list': ['n', 'n_t', 'n_x', 'n_xx', 'u', 'u_t', 'u_x', 'u_xx', 'ϵ', 'ϵ_t', 'ϵ_x'],
    'num_stages': 10
}
pinn = PINN(pars)


In [14]:
pinn.train()

Training...:   0%|          | 0/100000 [00:00<?, ?it/s]


Epoch: 0, Loss: 1.0539e+03
density_loss: 1.9098e-09, vorticity_loss: 3.4090e-12, tpe_loss: 3.1409e+03
initial_n_loss: 8.2539e+00, initial_u_loss: 1.4526e-03, initial_ϵ_loss: 6.4043e-05
boundary_n_loss: 1.2552e+01, boundary_u_loss: 1.4390e-03, boundary_ϵ_loss: 1.0799e-05


2024/08/25 20:16:30 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Training PINN with 10.00% of interior points
Epoch: 291, Loss: nan
density_loss: 7.8639e-11, vorticity_loss: 3.1813e-11, tpe_loss: nan
initial_n_loss: 7.4175e+00, initial_u_loss: 2.0249e-05, initial_ϵ_loss: 2.6989e-06
boundary_n_loss: 1.1902e+01, boundary_u_loss: 1.3032e-04, boundary_ϵ_loss: 3.2475e-06
loss is NaN, stopping training...


2024/08/25 20:16:50 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


## Debug

In [ ]:
pars = {
    'experiment_name': 'simple_pinn_multistage_training_v1',
    'layers': 8,
    'width': 128,
    'lr': 1e-4,
    'epochs': 10000,
    'eval_interval': 500,
    'interior_batch_size': 2048,
    'initial_batch_size': 2048,
    'boundary_batch_size': 2048,
    'x_min': 0.0,
    'x_max': 1.0,
    't_min': 0.0,
    't_max': 10000,
    'device': 'cuda',
    'plot_training_outputs': True,
    'plot_vars_list': ['n', 'n_t', 'n_x', 'n_xx', 'u', 'u_t', 'u_x', 'u_xx', 'ϵ', 'ϵ_t', 'ϵ_x'],
    'num_stages': 10
}

In [ ]:
import numpy as np

In [ ]:
list(range(0,pars['epochs'],pars['epochs']//pars['num_stages']))